↑プロジェクトトークンを挿入したら、実行を忘れずに!


# Db2 Magic Command Demo
# - Db2 %sql magic コマンド -

https://github.com/IBM/db2-jupyter



**実行は　セルを選択して**

- **Windows： Ctrl+Enter**
- **Mac: ⌘ (command) + Enter 　または Ctrl + Enter**



このnotebookはIBMCloud上のWatson Studioのnotebookで動くように書かれていますが、他の環境でも動作すると思いますが、一部Watson Studio特有のCodeがあります。<br>
**`IBM Cloud Watson Studioのみ`** と書いてある場合はWatson Studio特有のCodeです。<br>
逆に他の環境だと必須のCodeはコメントアウトして **`IBM Cloud Watson Studio以外`** と書いてありますので、他の環境はコメントアウト部分を必要に応じて実行してください。


### **`IBM Cloud Watson Studioのみ`**
Watson Studio上のnotebookからIBM Cloud Object Storage(ICOS)へのFileの読み書き - project-libを使う -の手順が前準備として必要です。

まずは1,2を実施お願いします。

- 1.[ Projectを開く](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#1-project%E3%82%92%E9%96%8B%E3%81%8F)
- 2.[ 前準備アクセス・トークンの作成](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#2-%E5%89%8D%E6%BA%96%E5%82%99%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E3%83%88%E3%83%BC%E3%82%AF%E3%83%B3%E3%81%AE%E4%BD%9C%E6%88%90)

その後、このnotebookに

- [5: プロジェクト・トークンの挿入](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#5-%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E3%83%88%E3%83%BC%E3%82%AF%E3%83%B3%E3%81%AE%E6%8C%BF%E5%85%A5)を実施してください。


一番上に挿入されたセルの実行は忘れずにお願いします。

## 1. 事前準備 
### 1.1. 最新の`ibm-db`パッケージをインストールします。
Watson Studioのnotebookには`ibm-db`パッケージはインストール済みですが、2022年1月16日現在パッケージが若干古いので最新の`ibm-db`パッケージに更新します。

またWatson Studioのnotebookでなくとも、Jupyter Notebook環境があれば、`ibm-db`パッケージをインストールすることでDb2のサポートを追加することができます。このパッケージはPythonに適切なデータベースコマンドを追加し、Db2データにアクセスできるようにします。<br>

以下を実行します。

In [ ]:
!pip install -U ibm_db

### 1.2. まずは**Db2 %sql magic コマンド**のモジュールをダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/kyokonishito/db2-jupyter/fix_createDF_varchar_length/db2.ipynb

### 1.2. ダウンロードした`db2.ipynb`を実行します。

このコマンドで**%sql**コマンドが使えるようになります！

In [ ]:
%run db2.ipynb

`IBM Cloud Watson Studioのみ`:警告メッセージはそのままで問題ありません。

qgridは試したところ、導入しても動きませんでした。
```
Warning: QGRID is unavailable for displaying results in scrollable windows.
         Install QGRID if you want to enable scrolling of result sets.
Warning: PANDAS level does not support Db2 typing which will can increase memory usage.
         Install PANDAS version 1.3+ for more efficient dataframe creation.
```

## 2. Db2 on cloudの情報をセット
### 2.1. Db2 on Cloudのサービス資格情報をコピペし、実行します。

Db2 on CloudではないDb2を使う場合は、ここは実行せず、接続情報は2.2で手打ちしてください。

**方法:**
[IBM Cloudコンソールから資格情報を作成・参照(Liteプランの場合はこれのみ)](https://qiita.com/nishikyon/items/69fee851849e7ef7e163#ibm-cloud%E3%82%B3%E3%83%B3%E3%82%BD%E3%83%BC%E3%83%AB%E3%81%8B%E3%82%89%E8%B3%87%E6%A0%BC%E6%83%85%E5%A0%B1%E3%82%92%E4%BD%9C%E6%88%90%E5%8F%82%E7%85%A7lite%E3%83%97%E3%83%A9%E3%83%B3%E3%81%AE%E5%A0%B4%E5%90%88%E3%81%AF%E3%81%93%E3%82%8C%E3%81%AE%E3%81%BF) の<br>
1, 2, 3, 5で内容を表示し、表示された内容を`service_credencials = \`の行の下にコピペしてください。

例:
```py
# @hidden_cell
#service_credencials = 
# <ここにコピペ>
service_credencials = \
{
  "connection": {
    "cli": {
      "arguments": [
      ・・・・
```



In [ ]:
# @hidden_cell
#service_credencials = 
# <ここにコピペ>
service_credencials = \


### 2.2 必要なパラメータをセット
Db2 on Cloudのサービス資格情報から取らずに、直接値を入れてもOKです。

Db2 on Cloudの以外のDb2を使う場合は、直接入れてください。

- databese: データベース名
- user: ユーザー名
- password: パスワード
- host: ホスト名　または　IPアドレス
- port: ポート番号

In [ ]:
databese = service_credencials["connection"]["db2"]["database"]
user = service_credencials["connection"]["db2"]["authentication"]["username"]
password = service_credencials["connection"]["db2"]["authentication"]["password"]
host =  service_credencials["connection"]["db2"]["hosts"][0]["hostname"]
port =  service_credencials["connection"]["db2"]["hosts"][0]["port"]

# 3. Db2 %sql Magic コマンドを使ってみる

以下が終わっていることが前提です。

- 1.事前準備
- 2.Db2 on cloudの情報をセット

## Magic コマンド
Jupyterには、Pythonコードや特別なアクションを代行して実行できる一連のMagicコマンドが用意されています。
例えば、%systemはシステムコマンドを発行します。


`%run db2.ipynb`コマンドは、**db2.ipynb**ファイルの内容を読み込み、新しい **%sql**と **%%sql**コマンドを作成します。

```
%run db2.ipynb
```


### 基本


`%sql` コマンドは一行コマンドに使用され、`%%sql` は SQL のブロック(複数のSQL)を実行します。

`%sql` コマンドの結果のみ変数に代入できます。

pythonの変数を`%sql`コマンドに渡すには，{}の中括弧で変数名を挟みます。

単一の %sql コマンドは、行末にバックスラッシュ`\`を使用して複数行に記載できます。

`%%sql` ブロック内の複数のステートメントでは、セミコロン`;`をデリミタとして使用します。

## 3.1 Db2 に接続
- SSL接続の場合は最後にSSL TRUEをつけます
 - Db2 on CloudはSSL接続必須なのでつけます
 
 ```
 %sql CONNECT TO <データベース名> USER <ユーザー名> USING <パスワード> HOST <ホスト名>　または　IPアドレス> port <ポート番号> SSL TRUE
 ```
 
 例: SSL接続
 ```
  %sql CONNECT TO bludb USER nishito USING P@ssw0rd HOST ibm.aaa.ccc.com port 50001 SSL TRUE
 ```
 例: SSL接続なし
 ```
  %sql CONNECT TO bludb USER nishito USING P@ssw0rd HOST ibm.aaa.ccc.com port 50000
 ```
 
 **以下の接続文は後述する文字列置換で2.2でセットしたパラメータを使っています。** <br>
 文字列置換については後ほど[6.文字列置換](#6.-文字列置換)で説明します。

In [ ]:
#2.2でセットした情報を使用 SSL不要の場合はSSL TRUEを削除
%sql CONNECT TO {databese} USER {user} USING {password} HOST {host} port {port} SSL TRUE

## 3.2 定義済みマクロ
### 1. table一覧取得
**LIST TABLES [FOR ALL | FOR SCHEMA name]** <br>
- 現在のスキーマまたはデータベースのテーブルを一覧表示
- 何もつけないとデフォルトスキーマが指定され、接続したユーザー名のスキーマのテーブルを一覧表示になります。

In [ ]:
%sql list tables

### 2. sample db作成 
**sampledata**<br>
Sample tables [EMPLOYEE, DEPARTMENT]を作成します。

スキーマ名はユーザーIDになります。

In [ ]:
%sql sampledata

もう一度データベースのテーブルを一覧表示してみます。

In [ ]:
%sql list tables

### 3. table情報取得
**DESCRIBE [TABLE]**<br>
テーブルの情報を表示する

In [ ]:
%sql DESCRIBE TABLE EMPLOYEE
%sql DESCRIBE TABLE DEPARTMENT

## 3.3 SQLを実行してみよう!

### 1. オプションなして、実行してみます：

In [ ]:
%sql select * FROM EMPLOYEE

### 2. 出力フォーマット
デフォルトではPandas Dataframe形式で出力されます!

出力の最初の5行と最後の5行を表示します。<br>
もっと見たい場合は、特別なフラグ(**-all**)を使用するか、いくつかの設定を変更します。<br>
(他に-gridフォーマットのオプションがありますが、Watson Studioのnotebookではエラーが出るのでここでは使用しません)



- **-all. -a** : 全部レコード表示
(ただし現状MAX 100行)

In [ ]:
%sql -all select * FROM EMPLOYEE

または表示する最大行数を変更できます。(ソースを読むと最大100でした)


```
%sql option MAXROWS 30
```

現在の値は`%sql option list `の出力の**MAXROWS**の値を参照してください。



In [ ]:
#現在の値
%sql option list

In [ ]:
#値を変更
%sql option MAXROWS 30

In [ ]:
%sql select * FROM EMPLOYEE

出力行数がMAXROWSより大きいと効かないようで、、、なんかいまいちな、、、。<br>
(pandas的にはMAXROS(pandas.options.display.max_rows)が出力より多い場合はpandas.options.display.min_rowsの値が使われるそうで、、、）<br>



結局MAX100行なら、**-all** を使うのがよいでしょう。<br>
100行超えた場合は、前後10行となります(いまいち)。

結局、細かに値を見たい場合は 結果をpandasのDataframeに入れてしまい、そこで操作するのが簡単です。<br>
python pandasの知識がちょっと必要になってきてしまいます。ちょっとなので覚えてしまえば簡単です。


参考:
- Dataframeについては　https://github.com/kyokonishito/thinkjapan2018/blob/master/jupyter_files/02_Dataframe_Sample.ipynb

- Dataframeの操作例は　https://github.com/kyokonishito/thinkjapan2018/blob/master/jupyter_files/DEMO_01_SELECT_DF.ipynb　を参考にしてみてください
   - (2018年Think Japanでの講演の一部です)。

### 3. SELECTの結果を変数に代入
結果セットをPythonの変数に割り当てることができます。

In [ ]:
df_employee = %sql select * FROM EMPLOYEE

変数に格納された後は、Pandasの構文を使って表示したり、スライスしたりすることができます。

In [ ]:
#単に表示 (前後5行)
df_employee

In [ ]:
# 0行目のFIRSTNMEを表示 (行数は0から開始)
df_employee['FIRSTNME'][0]

In [ ]:
#  index 10行目を表示 (0から始まるので実質11行目)
df_employee.iloc[[10]]

In [ ]:
# WORKDEPTがC01の行を表示 (これはSQLでもいいかも、、、)
df_employee.loc[(df_employee['WORKDEPT']=='C01')]

In [ ]:
#　最初10行
df_employee.head(10)

In [ ]:
#　最後10行
df_employee.tail(10)

### 4. SQL文とSQLブロック
複数のSQLを実行したい場合は場合は　`%%sql`を使います。　Cell内のすべてがDb2 SQLと設定されます(＃のコメントは使用できません)。 <br>
デフォルトではSQL文の区切りにセミコロン(;)が使われています。 <br>
デフォルトはAuto Commitです。変更したい場合は最後に紹介する参考資料を参照してください。

In [ ]:
%%sql
DROP TABLE IBM_DATABASE_DOJO;
CREATE TABLE IBM_DATABASE_DOJO (
    START TIMESTAMP,
    END TIMESTAMP,
    ROOM VARCHAR(8),
    TITLE VARCHAR(128),
    SPEAKER VARCHAR(128)
);
INSERT INTO IBM_DATABASE_DOJO VALUES
 ('2022-05-23-16.00.00.000000', '2022-05-23-17.00.00.000000', 'B', '無償のDb2 on CloudをJupyter Notebookから使ってみよう', 'Teruaki Niwa');

In [ ]:
#  作成したIBM_DATABASE_DOJO テーブルを見てみます
%sql select * from IBM_DATABASE_DOJO

### 5. パラメータ(変数)の使用


ステートメントにPython変数を与えるには、以下のフォーマットを使用します。 
**:var_name**
```
first_name = 'JOHN'
%sql SELECT * FROM EMPLOYEE WHERE FIRSTNME = :first_name
```
INのリストも対応しています
```
first_name = ["JOHN","SALLY","ROBERT"]
%sql SELECT * FROM EMPLOYEE WHERE FIRSTNME IN (:first_name)
```

In [ ]:
first_name = 'JOHN'
%sql SELECT * FROM EMPLOYEE WHERE FIRSTNME = :first_name

In [ ]:
first_name = ["JOHN","SALLY","ROBERT"]
%sql SELECT * FROM EMPLOYEE WHERE FIRSTNME IN (:first_name)

### 6. 文字列置換
- SQL構文の項目（列など）の値を指定するには、**{varname}**形式を使用します。
- 文字列のそのまま置き換えになります。
- 単一行の %sql コマンドでのみ使用できます。

```
colmns = 'EMPNO,LASTNAME'
first_name = "'JOHN'"
%sql SELECT {colmns} FROM EMPLOYEE WHERE FIRSTNME = {first_name}
```

In [ ]:
colmns = 'EMPNO,FIRSTNME, LASTNAME'
first_name = "'JOHN'"
%sql SELECT {colmns} FROM EMPLOYEE WHERE FIRSTNME = {first_name}

変数を使用した場合、発行されたsqlをみるには,echoオプションが便利です。
- **-e, -echo** : マクロの置換のエコー

In [ ]:
colmns = 'EMPNO,FIRSTNME, LASTNAME'
first_name = "'JOHN'"
%sql -e SELECT {colmns} FROM EMPLOYEE WHERE FIRSTNME = {first_name}

### 7. 簡易グラフ機能
%sqlコマンドでは、簡単なチャートが作成できます:
- -bar –棒グラフ
- -pie –円グラフ 
- -line –折れ線グラフ

In [ ]:
%sql -bar SELECT WORKDEPT, AVG(SALARY) FROM EMPLOYEE GROUP BY WORKDEPT

In [ ]:
%sql -pie SELECT WORKDEPT, AVG(SALARY) FROM EMPLOYEE GROUP BY WORKDEPT

In [ ]:
%sql -line SELECT WORKDEPT, AVG(SALARY) FROM EMPLOYEE GROUP BY WORKDEPT

###  8.  Dataframeからテーブル作成機能
(ちょっとpython pandasのcodeが必要)



`Jupyter notebook`<br>
Db2に入れるデータとして、
[神奈川県警察のオープンデータ](https://www.police.pref.kanagawa.jp/mes/mesd0145.htm) に掲載されている,<br>
 [自転車盗(令和２年発生)](https://www.police.pref.kanagawa.jp/csv/kanagawa_2020zitensyatou.csv)を取得します。

In [ ]:
!wget https://www.police.pref.kanagawa.jp/csv/kanagawa_2020zitensyatou.csv

In [ ]:
#データの中身を確認
!head kanagawa_2020zitensyatou.csv

`Python pandas`<br>
ダウンロードしたCSVファイルをpandasのDataFrameに入れます。
その際Dateのフォーマットをpandasのdateフォーマットにします。

In [ ]:
#ダウンロードしたCSVファイルをpandasのDataFrameに入れます。
import pandas as pd
!pip install japanize_matplotlib
import japanize_matplotlib

df_all = pd.read_csv('kanagawa_2020zitensyatou.csv', parse_dates=['発生年月日（始期）'],date_parser=lambda x: pd.to_datetime(x, format='%Y/%m/%d').date)
df_all = df_all.replace(['60-64歳','65-69歳'],60).replace(['歳代','歳以上','歳未満',],'',regex=True)
df_all = df_all[df_all['被害者の年齢'] != '法人・団体、被害者なし']
df_all['被害者の年齢'] = df_all['被害者の年齢'].astype(int)
display(df_all)

### Dataframeからテーブル作成

USING句を使って、PandasのDataframeからDb2のテーブルを作成します。
 
**sql USING dataframe [CREATE | APPEND | REPLACE] TABLE table [WITH DATA | COLUMNS ASIS | KEEP [INT64 |FLOAT64] |LIMIT x]**<br>

- **dataframe** -値を格納するdataframeの名前。
- **CREATE** -テーブルの作成 - テーブルが存在する場合は失敗。
- **REPLACE** -テーブルが存在しない場合は作成し、存在する場合は置換。
- **APPEND** -既存のテーブルにデータを追加。テーブルは作成しない。
- **table** -テーブル名
- **WITH DATA** - dataframeのデータをDb2テーブルにコピーする
- **COLUMNS ASIS** - Db2フレンドリーな名前ではなく、 dataframeのカラム名を使用する(訳注: Db2フレンドリーな名前は次ページ参照)
- **KEEP INT64, KEEP FLOAT64** - Db2 のフレンドリーな型の代わりに、データフレームから INT64 および FLOAT64 フォーマットを使用する。
- **LIMIT x** -テーブルにコピーする行数を "x "に制限する


In [ ]:
#念のためテーブルは削除してから,なければエラーが出るが無視
%sql drop table STOLEN_BICYCLE 

# Dataframeからテーブル作成
%sql using df_all create table STOLEN_BICYCLE with data

In [ ]:
df_select_reslut = %sql -all select * from STOLEN_BICYCLE
df_select_reslut

In [ ]:
%sql -bar SELECT "施錠関係", "被害者の年齢",count(*) as "罪名"from STOLEN_BICYCLE GROUP BY rollup("施錠関係","被害者の年齢")


### 9. SELECTした結果をWatsonStudioのプロジェクトのデータ資産に保存
(ちょっとpython pandasとwatson studio用のcodeが必要)
#### **`IBM Cloud Watson Studioのみ`**

In [ ]:
# 保存
project.save_data("kanagawa_2020zitensyatou.csv",df_select_reslut.to_csv(index=False),overwrite=True)

#### **`IBM Cloud Watson Studio以外`**

中身の確認はプロジェクトのデータ資産から！

In [ ]:
# ローカルに保存 (PCなどでJupyter Notebookを動かしている場合)
df_select_reslut.to_csv("kanagawa_2020zitensyatou.csv", index=False)
# 中身の確認
!head newly_confirmed_cases_daily.csv

### 10. 後片付け
#### 作成したテーブルのDROP

In [ ]:
%%sql 
drop table EMPLOYEE;
drop table DEPARTMENT;
drop table IBM_DATABASE_DOJO;
drop table STOLEN_BICYCLE;

In [ ]:
#　確認
%sql list tables

#### DB接続の切断

In [ ]:
# DB接続の切断
%sql connect close

# お疲れ様でした！　以上です。

もっといろいろなコマンドの使い方は<br>
こちらを参照：https://speakerdeck.com/kyokonishito/db2-and-jupyter-notebooks

または本家Github repositoryを参照してください(英語)<br>
最新の情報もこちらになります。
https://github.com/IBM/db2-jupyter